## Red-Black Tree  
http://software.ucv.ro/~mburicea/lab8ASD.pdf

In [0]:
import numpy as np

class Node:
  def __init__(self, key, color=0):
    self.parent = None
    self.left = None
    self.right = None
    self.color = color    # 0 = black, 1 = red
    self.key = key

In [0]:
class redBlackTreeMagic:
  def __init__(self):
    self._blackHeight = 0

  def _insertBST(self, parent, node):
    if parent != None:
      if parent.key > node.key:
        if parent.left == None:
          parent.left = node
          node.parent = parent
        else:
          self._insertBST(parent.left, node)
      else:
        if parent.right == None:
          parent.right = node
          node.parent = parent
        else:
          self._insertBST(parent.right, node)        
    else:
      parent = node

  def _getParent(self, node):
    if node == None:
      return None
    else:
      return node.parent

  def _getSibling(self, node):
    parent = self._getParent(node)
    if parent == None:
      return None

    if node == parent.left:
      return parent.right
    else:
      return parent.left

  def _getGrandParent(self, node):
    return self._getParent(self._getParent(node))

  def _getUncle(self, node):
    parent = self._getParent(node)

    return self._getSibling(parent)

  def _rotateRight(self, node):
    tmpNode = node.left
    parent = self._getParent(node)
    node.left = tmpNode.right

    tmpNode.right = node
    node.parent = tmpNode
    
    if node.left != None:
      node.left.parent = node

    if parent != None:
      if node == parent.left:
        parent.left = tmpNode
      elif node == parent.right:
        parent.right = tmpNode

    tmpNode.parent = parent
    return node

  def _rotateLeft(self, node):
    tmpNode = node.right
    parent = self._getParent(node)
    node.right = tmpNode.left

    tmpNode.left = node
    node.parent = tmpNode
    
    if node.right != None:
      node.right.parent = node


    if parent != None:
      if node == parent.left:
        parent.left = tmpNode
      elif node == parent.right:
        parent.right = tmpNode

    tmpNode.parent = parent
    return node

  def _redUncleRedParent(self, node):
    grandParentNode = self._getGrandParent(node)

    if grandParentNode == None:
      return

    grandParentNode.color = 0
    node = grandParentNode
    
    return node

  def _blackUncleRedParent(self, node, parent):
    # 2 cases here: 
    # - parent is a left child of grandparent
    # - parent is a right child of grandparent
    # Each case should consider is node a left child or right child of parent
    grandparent = self._getGrandParent(node)

    if grandparent == None:
      return

    if parent == grandparent.left:
      if node == parent.right:
        parent = self._rotateLeft(parent)
        node = parent
        parent = node.parent

      grandparent = self._rotateRight(grandparent)
      grandparent.color, parent.color = parent.color, grandparent.color
      node = parent
    else:
      if node == parent.left:
        parent = self._rotateRight(parent)
        node = parent
        parent = node.parent

      grandparent = self._rotateLeft(grandparent)
      grandparent.color, parent.color = parent.color, grandparent.color
      node = parent

    return node


  def _fixTree(self, node):
    uncleNode = self._getUncle(node)
    parent = self._getParent(node)

    # If node is root or parent is black --> nothing to do here: we are either root or everything is fine
    if parent == None or parent.color == 0 or node.color == 0:
      return

    # Red uncle. If uncle is None, then it is black
    if uncleNode != None and uncleNode.color == 1:
      uncleNode.color = 0
      parent.color = 0
      node = self._redUncleRedParent(node)
    else:
      node = self._blackUncleRedParent(node, parent)

    self._fixTree(node)

  def insertNewNode(self, parent, node):
    # BST insert algorithm used
    self._insertBST(parent, node)
    # Fix tree
    self._fixTree(node)

    return node

  def _replaceCurrent(self, node, leaf):
    if node == None:
      return

    leaf.parent = node.parent
    if node == node.parent.left:
      node.parent.left = child
    else:
      node.parent.right = child

    del node

  def _deleteIfSiblingIsBlack(self, node):
    child = self._getSibling(node)

    if node.parent.color == 0 and child.color == 0 and child.left.color == BLACK and child.right.color == 0:
      child.color = 1
      self._deleteIfNewRoot(node.parent)
    else:
      self._deleteIfOnlySiblingBlack(node)

  def _deleteOneBlackRightRed(self, node):
    child = self._getSibling(node)

    child.color = node.parent.color
    node.parent.color = 0

    if node == node.parent.left:
      child.right.color = 0
      self._rotateLeft(node.parent)
    else:
      child.left.color = 0
      self._rotateRight(node.parent)

  def _deleteOneBlackLeftRed(self, node):
    child = self._getSibling(node)

    if child.color == 0:
      if node == node.parent.left and child.right.color == 0 and child.left.color == 1:
        child.color = 1
        child.left.color = 0
        self._rotateRight(child)
      elif node == node.parent.right and child.left.color == 0 and child.right.color == 1:
        child.color = 1
        child.right.color = 0
        self._rotateLeft(child)

    self._deleteOneBlackRightRed(node)

  def _deleteIfOnlySiblingBlack(self, node):
    child = self._getSibling(node)

    if node.parent.color == 1 and child.color == 0 and child.left.color == 0 and child.right.color == 0:
      child.color = 1
      node.parent.color = 0
    else:
      self._deleteOneBlackLeftRed(node)

  def _deleteIfParentIsNotNone(self, node):
    child = self._getSibling(node)

    if child.color == 1:
      node.parent.color = 1
      child.color = 0

    if node == node.parent.left:
      self._rotateLeft(node.parent)
    else:
      self._rotateRight(node.parent)

    self._deleteIfSiblingIsBlack(node)

  def _deleteIfNewRoot(self, node):
    if node.parent != None:
      self._deleteIfParentIsNotNone(node)


  def deleteNode(self, node):
    if node.right != None:
      child = node.right
    else:
      child = node.left

    self._replaceCurrent(node, child)
    
    if node.color == 0:
      if child.color == 1:
        child.color = 0
      else:
        self._deleteIfNewRoot(child)

    del node

  def inorder(self, node):
    if node == None:
      return

    self.inorder(node.left)
    print(node.key)

    self.inorder(node.right)

  def preorder(self, node):
    if (node == None):
      return

    print(node.key)
    self.preorder(node.left)
    self.preorder(node.right)

  def postorder(self, node):
    if node == None:
      return

    self.postorder(node.left)
    self.postorder(node.right)
    print(node.key)

In [26]:
rbtm = redBlackTreeMagic()

root = rbtm.insertNewNode(None, Node(10))
rbtm.insertNewNode(root, Node(1, 1))
rbtm.insertNewNode(root, Node(5, 1))
rbtm.insertNewNode(root, Node(11, 1))
rbtm.insertNewNode(root, Node(7, 1))
rbtm.insertNewNode(root, Node(12, 1))
rbtm.insertNewNode(root, Node(15, 1))
rbtm.insertNewNode(root, Node(19, 1))
rbtm.insertNewNode(root, Node(17, 1))

print("Preorder")
rbtm.inorder(root)
print("Inorder")
rbtm.inorder(root)
print("Postorder")
rbtm.postorder(root)

Preorder
7
10
11
12
15
17
19
Inorder
7
10
11
12
15
17
19
Postorder
7
11
15
19
17
12
10
